The purpose of this notebook is to identify the frequency of conflicts between metrics that purport to measure the degree to which a plan favors one party or another.

In [ ]:
from typing import List, Dict, Any, Set

import os
import pandas as pd
from collections import defaultdict

from rdametrics import states, chambers, ensembles

Load the scores dataframe

In [ ]:
scores_path: str = "~/local/beta-ensembles/prepackaged/scores/scores.parquet"
scores_df = pd.read_parquet(os.path.expanduser(scores_path))

Helper code

In [ ]:
import pandas as pd

ensembles = [e for e in ensembles if e not in ["A1", "A2", "A3", "A4", "Rev*"]]

partisan_bias = {
    "partisan_advantage": ["disproportionality", "efficiency_gap"],
    "partisan_symmetry": ["seats_bias", "votes_bias", "geometric_seats_bias"],
    "packing_cracking": ["mean_median_average_district", "lopsided_outcomes", "declination"],
}

categories: List[str] = list(partisan_bias.keys())

def same_sign(a, b):
    if a == 0.0 or b == 0.0:
        return True
    
    if pd.isna(a) or pd.isna(b):
        return True
    
    assert isinstance(a, (int, float)) and isinstance(b, (int, float)), f"Unexpected types: {type(a)}, {type(b)}"

    return a * b > 0

def same_signs(values):
    # Filter out zero values and NaN values
    non_zero_defined_values = [x for x in values if x != 0 and not pd.isna(x)]
    
    # If no valid values or only one valid value, return True
    if len(non_zero_defined_values) <= 1:
        return True
    
    # Check if all valid values have the same sign
    all_positive = all(x > 0 for x in non_zero_defined_values)
    all_negative = all(x < 0 for x in non_zero_defined_values)
    
    return all_positive or all_negative

# def same_signs(list1, list2):
#     # Combine both lists and filter out zero values and NaN values
#     all_values = list1 + list2
#     non_zero_defined_values = [x for x in all_values if x != 0 and not pd.isna(x)]
    
#     # If no valid values or only one valid value, return True
#     if len(non_zero_defined_values) <= 1:
#         return True
    
#     # Check if all valid values have the same sign
#     all_positive = all(x > 0 for x in non_zero_defined_values)
#     all_negative = all(x < 0 for x in non_zero_defined_values)
    
#     return all_positive or all_negative

def partisan_balance(Vf: float) -> float:
    balance: float = Vf - 0.5
    if Vf < 0.5:
        balance = -balance
    return balance

Setup counters for various kinds of conflicts.

In [ ]:
from itertools import combinations
import copy

by_state: Dict[str, Any] = dict()

for xx in states:
    by_state[xx] = {
        "Vf": None,
        "total": 0
    }

conflicts = dict()
_ledger = {
    "counts": {xx: 0 for xx in states},
    "combos": set(),
    "example": None,
    "values": None,
    "delta": None,
}

cross_categories = list(combinations(categories, 2))
for c in categories + cross_categories:
    conflicts[c] = copy.deepcopy(_ledger)

# conflicts

Count instances where the scores conflict. Keep track of state / chamber / ensemble combinations.

In [ ]:
for index, row in scores_df.iterrows():
    if row["ensemble"] not in ensembles:
        continue

    xx, chamber, ensemble = (row["state"], row["chamber"], row["ensemble"])
    combo = (xx, chamber, ensemble)

    by_state[xx]["total"] += 1
    if by_state[xx]["Vf"] is None:
        by_state[xx]["Vf"] = row["estimated_vote_pct"]

    consistent = {
        "partisan_advantage": True,
        "packing_cracking": True,
        "partisan_symmetry": True
    }

    # Check consistency within each category

    for c in categories:
        values = [row[m] for m in partisan_bias[c]]
        delta = abs(max(values) - min(values))
        if not same_signs(values):
            consistent[c] = False
            conflicts[c]["counts"][xx] += 1
            conflicts[c]["combos"].add(combo)
            if conflicts[c]["example"] is None or delta > conflicts[c]["delta"]:
                conflicts[c]["example"] = combo
                conflicts[c]["values"] = values
                conflicts[c]["delta"] = delta

    # Compare consistency across categories
    
    for cat1, cat2 in cross_categories:
        if consistent[cat1] and consistent[cat2]:
            v1 = row[partisan_bias[cat1][0]]
            v2 = row[partisan_bias[cat2][0]]
            if not same_sign(v1, v2):
                conflicts[(cat1, cat2)]["counts"][xx] += 1
                conflicts[(cat1, cat2)]["combos"].add(combo)
                if conflicts[(cat1, cat2)]["example"] is None:
                    conflicts[(cat1, cat2)]["example"] = combo

# (by_state, conflicts)

Aggregate the results into a reportable format.

In [ ]:
report = dict()

for c, _ledger in conflicts.items():
    report[c] = dict()

    diffs_by_state = [(xx, partisan_balance(by_state[xx]['Vf']), count / by_state[xx]['total']) for xx, count in _ledger["counts"].items()]
    diffs_by_state.sort(key=lambda x: x[1])

    diffs = [(xx, f"{Vf:.2%}", f"{rate:.2%}") for xx, Vf, rate in diffs_by_state]

    report[c] = {
        "conflict-rate": diffs,
        "compare-rate": compare,
        "combos": len(_ledger["combos"]),
        "example": _ledger["example"],
        "values": _ledger["values"],
        "delta": _ledger["delta"],
    }

# report

In [ ]:
for _category, _ledger in report.items():
    print(f"Category: {_category}")
    print(f"  {len(_ledger['conflict-rate'])} states with conflicts / {_ledger['combos']} conflicting combos")
    print(f"  State, Deviation from 50-50 Balance, Conflict Rate")
    print(f"  { _ledger["conflict-rate"]}")
    if _ledger["example"] is not None:
        xx, chamber, ensemble = _ledger["example"]
        print(f"  Example conflict: {xx} / {chamber} / {ensemble}")
        print(f"  Values = {_ledger['values']}  (delta = {_ledger['delta']})")
    print()